As described by @cdeotte [here](https://www.kaggle.com/c/santa-2021/discussion/288995), we can solve Santa 2021 as a routing problem. Here we solve it with Google OR-Tools using 3 vehicles so we will get 3 different routes. Then we convert each route to string to create the submission csv filve.

In [ ]:
import itertools
import pandas as pd
import numpy as np

In [ ]:
mandatory = ['54'+''.join(x) for x in itertools.permutations(['1','2','3','6','7'], 5)]

non_mandatory = []
all_permu = [''.join(x) for x in itertools.permutations(['1','2','3','4','5','6','7'], 7)]
for permu in all_permu:
    if permu not in mandatory:
        non_mandatory.append(permu)

In [ ]:
all_permu = mandatory + mandatory + mandatory + non_mandatory

In [ ]:
size_dict = len(non_mandatory) + 3*len(mandatory)
dist = np.empty((size_dict, size_dict), dtype=int)

In [ ]:
for k, permu in enumerate(all_permu):
    for l, permu2 in enumerate(all_permu):
        if permu == permu2:
            continue
        distance = 7
        for i in range (1,7):
            if permu2[:i] == permu[-i:]:
                distance = 7 - i
        dist[k,l] = distance

In [ ]:
dist

In [ ]:
"""Vehicles Routing Problem (VRP)."""

from ortools.constraint_solver import pywrapcp


def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dist
    data['num_vehicles'] = 3
    data['starts'] = [0, 120, 240]
    data['ends'] = [0, 120, 240]
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

def solve():
    """Solve the VRP problem."""
    
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['starts'],
                                           data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """"Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)

    # Set default search parameters.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.time_limit.seconds = 300

    # set mandatory locations SetAllowedVehiclesForIndex
    for location_index in range(len(mandatory)):
        routing.SetAllowedVehiclesForIndex([0], location_index) # vehicle 0
    for location_index in range(len(mandatory), 2*len(mandatory)):
        routing.SetAllowedVehiclesForIndex([1], location_index) # vehicle 1
    for location_index in range(2*len(mandatory), 3*len(mandatory)):
        routing.SetAllowedVehiclesForIndex([2], location_index) # vehicle 2
    
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    # Print solution on console.
    routes = False
    if solution:
        print('Solution after search:')
        print_solution(data, manager, routing, solution)
        routes = get_routes(solution, routing, manager)
        
    return routes

In [ ]:
routes = solve()

In [ ]:
def get_string(my_route):
    string = ''
    for i, item in enumerate(my_route):
        if i == 0:
            string0 = all_permu[item]
        else:
            item_before = my_route[i-1]
            dist_range = dist[item_before, item]
            add_string = all_permu[item][-dist_range:]
            string += add_string
    print('length', len(string))
    return string

In [ ]:
if routes:
    string1 = get_string(routes[0])
    string2 = get_string(routes[1])
    string3 = get_string(routes[2])
else:
    print ('No routes')
    string1 = ''
    string2 = ''
    string3 = ''

### Verify

In [ ]:
all_permutations = [''.join(x) for x in itertools.permutations(['1','2','3','4','5','6','7'], 7)]

for p in all_permutations:
    if p not in string1 and p not in string2 and p not in string3:
        print(p)

In [ ]:
for p in mandatory:
    if p not in string1:
        print('string1', p)
    if p not in string2:
        print('string2', p)
    if p not in string3:
        print('string3', p)

### Write Submission CSV

In [ ]:
# Convert numbers to emojis
replace_dict = {
 '5': '🎅',
 '4': '🤶',
 '8': '🌟',
 '1': '🦌',
 '2': '🧝',
 '3': '🎄',
 '6': '🎁',
 '7': '🎀'}

for k,v in replace_dict.items():
    string1 = string1.replace(k, v)
    string2 = string2.replace(k, v)
    string3 = string3.replace(k, v)

In [ ]:
sub = pd.DataFrame()
sub['schedule'] = [string1, string2, string3]
sub.to_csv('submission.csv',index=False)
sub.head()